## 当前环境确认
确认conda env / 所在目录 / 显卡信息

In [1]:
!conda env list
!pwd
!nvidia-smi
!export

# conda environments:
#
base                     /root/anaconda3
huggingface              /root/anaconda3/envs/huggingface
modelscope            *  /root/anaconda3/envs/modelscope

/root
Sun Jun  9 11:28:41 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     Off |   00000000:01:00.0 Off |                  N/A |
|  0%   32C    P8              8W /  165W |  

## swift 环境部署

In [9]:
!git clone https://github.com/modelscope/swift.git
!cd swift
%pip install -e '.[llm]' -i https://mirror.baidu.com/pypi/simple --trusted-host mirror.baidu.
%pip install 'ms-swift[llm]' -U -i https://mirror.baidu.com/pypi/simple --trusted-host mirror.baidu.


%pip install vllm -i https://mirror.baidu.com/pypi/simple --trusted-host mirror.baidu.com

fatal: destination path 'swift' already exists and is not an empty directory.
Looking in indexes: https://mirror.baidu.com/pypi/simple
Obtaining file:///root
ERROR: file:///root does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirror.baidu.com/pypi/simple
  Using cached https://mirror.baidu.com/pypi/packages/54/22/9f4238ac04f3e80825ec3bcbda3b89e6305087d8d6c6a8d0d3e5fc4d7f00/ms_swift-2.1.0-py3-none-any.whl (523 kB)
  Using cached https://mirror.baidu.com/pypi/packages/f0/62/9ebaf1fdd3d3c737a8814f9ae409d4ac04bc93b26a46a7dab456bb7e16f8/accelerate-0.31.0-py3-none-any.whl (309 kB)
  Using cached https://mirror.baidu.com/pypi/packages/21/0f/cf0943f4f55f0fbc7c6bd60caf1343061dff818b02af5a0d444e473bb78d/dacite-1.8.1-py3-none-any.whl (14 kB)
  Using cached jieba-0.42.1-py3-none-any.whl
  Using cached https://mirror.baidu.com/pypi/packages/80/3e/a59c0bcb192e4b26d253d

## 部署glm4v-9b-chat

In [3]:
# 多模态缺失下面pip modal
%pip install torchvision -i https://mirror.baidu.com/pypi/simple --trusted-host mirror.baidu.

Looking in indexes: https://mirror.baidu.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:

# !CUDA_VISIBLE_DEVICES=0 swift infer --model_type glm4v-9b-chat
# !swift deploy --host 10.0.0.17 --model_type qwen-7b-chat
# !CUDA_VISIBLE_DEVICES=0 swift deploy --host 10.0.0.17 --model_type qwen-7b-chat
!CUDA_VISIBLE_DEVICES=0 swift deploy --host 10.0.0.17 --model_type glm4v-9b-chat

OSError: Background processes not supported.

# swift multimodal 调优

## 自定义数据集 dataset

In [3]:
!pwd
# !CUDA_VISIBLE_DEVICES=0 swift sft \
#     --model_type glm4v-9b-chat \
#     --dataset /root/data/neusoft_cz_demo.json 

!CUDA_VISIBLE_DEVICES=0 swift sft \
    --model_type glm4v-9b-chat \
    --dataset coco-en-2-mini 

/root
run sh: `python /root/anaconda3/envs/modelscope/lib/python3.10/site-packages/swift/cli/sft.py --model_type glm4v-9b-chat --dataset coco-en-2-mini`
2024-06-09 13:09:18,177 - modelscope - INFO - PyTorch version 2.3.1 Found.
2024-06-09 13:09:18,177 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-06-09 13:09:18,200 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 91d2453206645ee6756bd6e31af60918 and a total number of 980 components indexed
[INFO:swift] Successfully registered `/root/anaconda3/envs/modelscope/lib/python3.10/site-packages/swift/llm/data/dataset_info.json`
[INFO:swift] Start time of running main: 2024-06-09 13:09:18.620359
[INFO:swift] Setting template_type: glm4v
[INFO:swift] Setting args.lazy_tokenize: True
[INFO:swift] Setting args.dataloader_num_workers: 1
[INFO:swift] output_dir: /root/output/glm4v-9b-chat/v10-20240609-130918
[INFO:swift] args: SftArguments(model_type='glm4v-9b-chat', model_id_

In [2]:
!WEBUI_SERVER=10.0.0.17 swift web-ui

[INFO:swift] Setting args.model_type: llava1_6-mistral-7b-instruct
[INFO:swift] Setting args.lazy_tokenize: True
[INFO:swift] Setting args.dataloader_num_workers: 1
[INFO:swift] output_dir: /root/output/llava1_6-mistral-7b-instruct/v4-20240609-125033/llava1_6-mistral-7b-instruct/v0-20240609-125617
[INFO:swift] Run training: CUDA_VISIBLE_DEVICES=0  nohup swift sft --model_id_or_path "AI-ModelScope/llava-v1.6-mistral-7b" --template_type "llava-mistral-instruct" --dataset coco-en-2-mini --save_steps "500" --output_dir "/root/output/llava1_6-mistral-7b-instruct/v4-20240609-125033" --lora_target_modules q_proj k_proj v_proj --learning_rate "1e-4" --gradient_accumulation_steps "16" --eval_batch_size "1" --add_output_dir_suffix False --output_dir /root/output/llava1_6-mistral-7b-instruct/v4-20240609-125033/llava1_6-mistral-7b-instruct/v0-20240609-125617 --logging_dir /root/output/llava1_6-mistral-7b-instruct/v4-20240609-125033/llava1_6-mistral-7b-instruct/v0-20240609-125617/runs  > /root/outp